In [1]:
import os
from ilastik.experimental.api import from_project_file
import numpy as np
from xarray import DataArray
# Add your imports here, e.g. for loading and preprocessing data
import imageio
from matplotlib import pyplot as plt
import tifffile
from tqdm import tqdm
import warnings
import pickle
warnings.filterwarnings('ignore')

In [ ]:
# instance = "PT"

# model_base = f"/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik/{instance}_adapted"
# image_base = f"/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik/{instance}/training/DIMR"
# exported_base = f"/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik/{instance}/training/DIMR_Ilastik/"
# models = [os.path.join(model_base, x) for x in os.listdir(model_base) if not x.startswith(".")]
# samples = [x.split("/")[-1].split(".")[0] for x in models if not x.startswith(".")]
# out = f"/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik/Evaluation/comparison"

In [ ]:
# for model, sample in zip(tqdm(models), samples):
    
#     print(model, sample)
    
#     pipeline = from_project_file(model)
    
#     sample_image_base = os.path.join(image_base, sample)
    
#     images = os.listdir(sample_image_base)

#     chosen = np.random.choice(images, 5)
    
#     for image in chosen:
        
#         corresponding = [x for x in os.listdir(os.path.join(exported_base, sample)) if image.split(".")[0] in x][0]
        
#         marker_image = DataArray(tifffile.imread(os.path.join(sample_image_base, image)), dims=("y", "x"))
#         exported_image = DataArray(tifffile.imread(os.path.join(exported_base, sample, corresponding)), dims=("y", "x"))
        
#         prediction = pipeline.predict(marker_image)
               
#         diff = float(np.abs((np.round(prediction[..., 0],0)-exported_image)).max())
        
#         fig, ax = plt.subplots(1, 4, figsize=(30, 10))
        
#         ax[0].imshow(prediction[..., 1])
#         ax[0].set_title("Prediction Probability")
#         ax[1].imshow(np.round(prediction[..., 1], 0).astype(bool))
#         ax[1].set_title("Thresholded")
#         ax[2].imshow(exported_image)
#         ax[2].set_title("Exported Binary")
#         ax[3].imshow(np.round(prediction[..., 1], 0).astype(bool)-exported_image)
#         ax[3].set_title("Difference")
#         plt.tight_layout()
        
#         if not os.path.isdir(f"{out}/{instance}/{sample}"):
                
#             os.mkdir(f"{out}/{instance}/{sample}")
        
#         plt.savefig(f"{out}/{instance}/{sample}/{image.split('.')[0]}")
        
#         del fig
#         del ax

In [2]:
instance = "BM"

model_base = f"/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik/{instance}/Ilastik_models"

simple_segmentation = True

for dir_type in ["other", "training"]:

    image_base = f"/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik/{instance}/{dir_type}/DIMR"
    models = [os.path.join(model_base, x) for x in os.listdir(model_base) if not x.startswith(".")]
    samples = [x.split("/")[-1].split(".")[0] for x in models if not x.startswith(".")]

    for model, sample in zip(models, samples):
        
        print(model, sample)
        
        pipeline = from_project_file(model)
        
        sample_image_base = os.path.join(image_base, sample)
        
        images = [x for x in os.listdir(sample_image_base) if not x.startswith(".")]
        
        if simple_segmentation:
            ss_out_dir = f"/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik/{instance}/{dir_type}/DIMR_ilastik_simplesegmentation/{sample}"
            if not os.path.isdir(ss_out_dir):  
                os.makedirs(ss_out_dir)
                
        out_dir = f"/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik/{instance}/{dir_type}/DIMR_ilastik_probabilities/{sample}"
        if not os.path.isdir(out_dir):
            os.makedirs(out_dir)
        
        for image in tqdm(images):
            
            ss_name = f"{image.split('.')[0]}_Simple Segmentation.tiff"
            prob_name = f"{image.split('.')[0]}_Probabilities.tiff"
            
            if os.path.isfile(os.path.join(out_dir, prob_name)) and os.path.isfile(os.path.join(ss_out_dir, ss_name)):
                #print("Skipping: ", image)
                continue
                
            marker_image = DataArray(tifffile.imread(os.path.join(sample_image_base, image)), dims=("y", "x"))
            
            prediction = pipeline.predict(marker_image)
            
            if simple_segmentation:
                
                if not os.path.isfile(os.path.join(ss_out_dir, ss_name)):
                    tifffile.imwrite(os.path.join(ss_out_dir, ss_name), np.round(prediction[..., 0].to_numpy(), 0))

            if not os.path.isfile(os.path.join(out_dir, prob_name)):
                tifffile.imwrite(os.path.join(out_dir, prob_name), prediction[..., 0].to_numpy())               


/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik/BM/Ilastik_models/153Eu-CD45_Eu153.ilp 153Eu-CD45_Eu153


100%|██████████| 421/421 [00:01<00:00, 217.42it/s]


/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik/BM/Ilastik_models/172Yb-IDO_Yb172.ilp 172Yb-IDO_Yb172


100%|██████████| 421/421 [00:01<00:00, 243.99it/s]


/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik/BM/Ilastik_models/142Nd-CD11b_Nd142.ilp 142Nd-CD11b_Nd142


...

100%|██████████| 150/150 [00:00<00:00, 249.57it/s]


/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik/BM/Ilastik_models/171Yb-CD4_Yb171.ilp 171Yb-CD4_Yb171


100%|██████████| 150/150 [00:00<00:00, 255.14it/s]


/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik/BM/Ilastik_models/155Gd-GD2_Gd155.ilp 155Gd-GD2_Gd155


100%|██████████| 150/150 [00:00<00:00, 250.09it/s]


In [ ]:
file_list = []
for root, dirs, files in os.walk("/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik/BM/other/DIMR_ilastik_probabilities"):
    for name in files:
        if not "probabilities" in name.lower():
            file_list.append(os.path.join(root, name))

In [ ]:
for file in file_list:
    os.remove(file)